In [16]:
import pandas as pd
import os
import numpy as np
import json
from haversine import haversine
import datetime
# load the zip centroids
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
# read stores from the reqeust
df_stores_quad_3=pd.read_excel("./Big Lots Q2 l Persistent Quad 3 Stores.xlsx",dtype=str,skiprows=3)
df_stores_quad_3=df_stores_quad_3[df_stores_quad_3.iloc[:,0]!="TOTAL STORES"]
df_stores_quad_3_Legacy=df_stores_quad_3[['Legacy Stores']].drop_duplicates()
df_stores_quad_3_SOTF=df_stores_quad_3[['SOTF Stores']].drop_duplicates()

df_stores_quad_3_Legacy.columns=['location_id']
df_stores_quad_3_SOTF.columns=['location_id']

df_stores_quad_3_Legacy['Store_Type']='Legacy'
df_stores_quad_3_SOTF['Store_Type']='SOTF'

df_stores_quad_3=df_stores_quad_3_Legacy.append(df_stores_quad_3_SOTF)
df_stores_quad_3=df_stores_quad_3[pd.notnull(df_stores_quad_3['location_id'])]

In [3]:
# read newest stroe list
store_list_20200501=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200501-152115-660.txt",
                               dtype=str,sep="|")
store_list_20200501['latitude_meas']=store_list_20200501['latitude_meas'].astype(float)
store_list_20200501['longitude_meas']=store_list_20200501['longitude_meas'].astype(float)

store_list_20200501=store_list_20200501[['location_id','open_dt','latitude_meas','longitude_meas']]
store_list_20200501=store_list_20200501[~store_list_20200501['location_id'].isin(['145','6990'])]
store_list_20200501.head(2)

,location_id,open_dt,latitude_meas,longitude_meas
0,3,1988-10-31,33.421570,-82.018921
1,30,1984-04-05,41.363132,-83.650909


In [4]:
# load the old trade area info

TA_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
TA_zips=TA_zips.parse("view_by_store",dtype=str)
TA_zips.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [5]:
# reshape to a structured long format
df_temporary=TA_zips[['location_id','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10']]
df_zip_by_store=pd.DataFrame()

for ind,row in df_temporary.iterrows():
    location_id=row['location_id']
    P_zips=eval(row['trans_P_zips_70_within_TA'])
    S_zips=eval(row['trans_S_zips_70_within_TA'])
    zip_10=eval(row['zips_in_10'])
    

    df_P=pd.DataFrame(zip([location_id]*len(P_zips),P_zips))
    if len(df_P)>0:
        df_P.columns=['location_id','zip_cd']
        df_P['zip_type']="P"
        
    df_S=pd.DataFrame(zip([location_id]*len(S_zips),S_zips))
    if len(df_S)>0:
        df_S.columns=['location_id','zip_cd']
        df_S['zip_type']="S"
    
    df_10=pd.DataFrame(zip([location_id]*len(zip_10),zip_10))
    if len(df_10)>0:
        df_10.columns=['location_id','zip_cd']
        df_10['zip_type']="zip_10"
    
    df_zip_by_store=df_zip_by_store.append(df_P).append(df_S).append(df_10)
    
df_zip_by_store.head(3)

,location_id,zip_cd,zip_type
0,1949,75040,P
1,1949,75098,P
2,1949,75044,P


In [6]:
# find out which is new, and which were covered in the TA
df_store_list=pd.merge(store_list_20200501,df_zip_by_store,on="location_id",how="left")

df_store_list_existing=df_store_list[pd.notnull(df_store_list['zip_type'])]
df_store_list_new=df_store_list[pd.isnull(df_store_list['zip_type'])]

In [7]:
del df_store_list_new['zip_cd']
del df_store_list_new['zip_type']

df_store_list_new.head(2)

,location_id,open_dt,latitude_meas,longitude_meas
15294,4679,2019-07-03,42.723314,-84.424187
15604,4702,2019-09-19,31.789767,-106.405255


In [8]:
# Check and found out 1 store with the latitude_meas == 0 , means that the data is incorrect due to the new open 
# Go to google map find the lat/long based on the address

# 5416, from google map
# 260 Remount Rd, Front Royal, VA 22630
# 38.9092607,-78.1831743

df_store_list_new.loc[df_store_list_new['location_id']=="5416","latitude_meas"]=38.9092607
df_store_list_new.loc[df_store_list_new['location_id']=="5416","longitude_meas"]=-78.1831743


/home/jian/.local/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
# Fetch the zip in 10 miles for the new stores 
df_store_list_new=df_store_list_new.reset_index()
del df_store_list_new['index']
df_store_list_new.head(3)

,location_id,open_dt,latitude_meas,longitude_meas
0,4679,2019-07-03,42.723314,-84.424187
1,4702,2019-09-19,31.789767,-106.405255
2,4704,2019-08-08,29.805086,-94.986416


In [10]:
df_new_store_10_zips=pd.DataFrame()
for i, row in df_store_list_new.iterrows():
    store_coor=[row['latitude_meas'],row['longitude_meas']]
    list_zip_by_store=[]
    for zip_cd, v in zip_centers.items():
        dist=haversine(store_coor,v,unit="mi")
        if dist<=10:
            list_zip_by_store.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_zip_by_store},index=['zip_10']*len(list_zip_by_store)).reset_index()
    df=df.rename(columns={"index":"zip_type"})
    df['location_id']=row['location_id']
    df_new_store_10_zips=df_new_store_10_zips.append(df)
df_store_list_new=pd.merge(df_store_list_new,df_new_store_10_zips,on="location_id",how="left")         

In [11]:
df_store_list_new.shape,df_store_list_new['location_id'].nunique()

((857, 6), 36)

In [12]:
# append together to have the full zips in TA for the newest (0501) storelist
df_store_list=df_store_list_existing.append(df_store_list_new)
df_store_list.head(5)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,latitude_meas,location_id,longitude_meas,open_dt,zip_cd,zip_type
0,33.42157,3,-82.018921,1988-10-31,30906,P
1,33.42157,3,-82.018921,1988-10-31,30904,S
2,33.42157,3,-82.018921,1988-10-31,30815,S
3,33.42157,3,-82.018921,1988-10-31,30901,S
4,33.42157,3,-82.018921,1988-10-31,30909,S


In [13]:
# split based on the planner list
df_quandrant_3_zips=df_store_list[df_store_list['location_id'].isin(df_stores_quad_3['location_id'].tolist())]
df_rest_zips=df_store_list[~df_store_list['location_id'].isin(df_stores_quad_3['location_id'].tolist())]

In [14]:
# get the zips only for the planner list but not in any other store TA
df_output_zips_only_for_quandrant_3=df_quandrant_3_zips[~df_quandrant_3_zips['zip_cd'].isin(df_rest_zips['zip_cd'].tolist())]
print(df_output_zips_only_for_quandrant_3.shape,df_output_zips_only_for_quandrant_3['zip_cd'].nunique())
df_output_zips_only_for_quandrant_3.head(4)

(442, 6) 412


,latitude_meas,location_id,longitude_meas,open_dt,zip_cd,zip_type
121,40.397225,132,-86.871866,1986-02-21,47909,P
122,40.397225,132,-86.871866,1986-02-21,47905,P
123,40.397225,132,-86.871866,1986-02-21,47906,S
124,40.397225,132,-86.871866,1986-02-21,46041,S


In [17]:
writer=pd.ExcelWriter("./BL_zips_for_2019_Quad_3_Stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_zips_only_for_quandrant_3.to_excel(writer,"zips_for_quad_3_only",index=False)
writer.save()